In [2]:
from typing import (Tuple,List,Dict,Union)
import pandas as pd
import numpy as np

from torch.nn import (Tanh,ReLU)
import torch

In [55]:
def dynamics(y_t:torch.tensor,sigma):
    
    a,b = y_t
    y_next = torch.zeros_like(y_t)
    
    relu = ReLU()

    y_next[0] =  1.0 * torch.tanh(a) + 0.8 * a * b + 1.0 * b - 1.0 * relu(torch.minimum(a,b)) + sigma * torch.randn(1)
    y_next[1] = 0.6 * torch.sign(b) + 0.5 * a * b - 1.0 * torch.maximum(a,b) + sigma * torch.randn(1)
    
    return y_next

In [56]:
sigma_true = 0.1

T_total = 2000
y = torch.zeros((2, T_total))
y_without_noise = torch.zeros((2, T_total))

y[:,0] = torch.tensor([0.5,0.5])
y_without_noise = torch.tensor([0.5,-0.5])

name_stock = dynamics(y,sigma_true)
noise = dynamics(y,0)
Nomura = name_stock[0]
PFN = name_stock[1]
# for t in range(1,T_total):
    
#     y[:,t] = dynamics(y[:,t-1],sigma_true)
#     y_without_noise[:,t] = dynamics(y[:,t-1],0.)

tensor([[ 0.6641,  0.0020,  0.0020,  ...,  0.0020,  0.0020,  0.0020],
        [ 0.0895, -0.1355, -0.1355,  ..., -0.1355, -0.1355, -0.1355]])

In [61]:
y[0,1:]

tensor([0., 0., 0.,  ..., 0., 0., 0.])